In [ ]:
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from typing import List

import os, sys
rootpath = ".."
sys.path.insert(0, f"{os.getcwd()}/{rootpath}/models")
import model_base
import model_weightinit
import model_dense_ft

degree_sign = u'\N{DEGREE SIGN}'

In [ ]:
ft_percents = [0.0, 0.2, 0.4, 0.6, 0.8]
shuffle_seeds = [17, 42, 64, 76, 98]

In [ ]:
def plot_multiseed_error_pir(ft_percents: List[float], from_building: str, from_tower: int, from_season: str, to_building: str, to_tower: int, to_season: str = None):
    """
    Performs the following steps
    1. Performs transfer learning by finetuning entire model
    2. Performs transfer learning by finetuning only the output layer
    3. Plots mean absolute errors for test datasets picked with 5 different random seeds for both transfer learning methods
    4. Compares errors from transfer with error in the base model of the tower being transferred to
    5. Plots performance improvement ratio
    """
    
    # RUN MODEL AND MAKE PREDICTIONS

    rmse_list = [[], []]
    mae_list = [[], []]
    # [transfer type index] [seed index] [error]
    # transfer type index 0: finetuning lstm and dense
    # transfer type index 1: finetuning only dense

    to_features = [f'{to_building}_Tower_{to_tower} enteringWaterTemp', 
                    f'{to_building}_Tower_{to_tower} outdoorAirDryBulb', 
                    f'{to_building}_Tower_{to_tower} outdoorAirWetBulb', 
                    f'{to_building}_Tower_{to_tower} vfdPercent', 
                    f'{to_building}_Tower_{to_tower} fanA_vfdPower', 
                    f'{to_building}_Tower_{to_tower} fanB_vfdPower', 
                    f'{to_building}_Tower_{to_tower} dayOfWeek', 
                    f'{to_building}_Tower_{to_tower} hourOfDay', 
                    f'{to_building}_Tower_{to_tower} efficiency']
    to_target = f'{to_building}_Tower_{to_tower} leavingWaterTemp'


    for s, seed in enumerate(shuffle_seeds):
        rmse_list[0].append([])
        rmse_list[1].append([])
        mae_list[0].append([])
        mae_list[1].append([])

        for ft_percent in ft_percents:
            # finetuning all layers
            rmse_wi, fig_wi, mae_wi = model_weightinit.weight_init_lstmdense_transfer(from_building_name=from_building, from_tower_number=from_tower, from_season=from_season, to_building_name=to_building, to_tower_number=to_tower, to_features=to_features, to_target=to_target, to_season=to_season, finetuning_percentage=ft_percent, display_results=False, use_delta=True, shuffle_seed=seed)
            rmse_list[0][s].append(rmse_wi)
            mae_list[0][s].append(mae_wi)

            # finetuning only output layer
            rmse_fe, fig_fe, mae_fe = model_dense_ft.weight_init_dense_transfer(from_building_name=from_building, from_tower_number=from_tower, from_season=from_season, to_building_name=to_building, to_tower_number=to_tower, to_features=to_features, to_target=to_target, to_season=to_season, finetuning_percentage=ft_percent, display_results=False, use_delta=True, shuffle_seed=seed)
            rmse_list[1][s].append(rmse_fe)
            mae_list[1][s].append(mae_fe)




    # PLOT MAE ERROR GRADIENT

    base_errors = pd.read_csv(f"{rootpath}/data/results/base_model_errors.csv", index_col="building-tower-season").loc[f"{to_building}{to_tower}_{to_season}"]

    # Create a subplots figure with two subplots and set title
    figfinal = make_subplots(rows=1, cols=2, subplot_titles=("Finetuning all layers (LSTM+Dense)", "Finetuning output layer only (Dense)"))
    figfinal.update_layout(
        title=f"{from_building} Tower {from_tower} ({from_season}) to {to_building} Tower {to_tower} ({to_season}): Comparison of transfer methods (MAE)",
        showlegend=True
    )

    # Add traces to the subplots
    for tranfer_method_i, tranfer_method in enumerate(["(left)", "(right)"]):
        row = 1
        col = tranfer_method_i + 1
        x = [fp * 100 for fp in ft_percents]
        # plot lines over each other for different seeds
        for s, seed in enumerate(shuffle_seeds):
            # add errors of model transfers
            figfinal.add_trace(
                go.Scatter(
                    x=x, 
                    y=mae_list[tranfer_method_i][s], 
                    name=f"MAE seed-{seed} {tranfer_method}"),
                row=row, col=col
            )
            # add error of base model
            figfinal.add_trace(go.Scatter(
                x=x, 
                y=[base_errors["MAE"]]*len(ft_percents), 
                name=f"Base model MAE seed-42", 
                line=dict(color='rgb(155, 185, 155)'),
                mode='lines'),
                row=row, col=col)
            # add standard deviation of errors for base model
            y_upper = [base_errors["MAE"] + base_errors["MAE_SD"]]*len(ft_percents)
            y_lower = [base_errors["MAE"] - base_errors["MAE_SD"]]*len(ft_percents)
            figfinal.add_trace(go.Scatter(
                x=x+x[::-1], # x, then x reversed
                y=y_upper+y_lower[::-1], # upper, then lower reversed
                fill='toself',
                fillcolor='rgba(155, 185, 155,0.1)',
                line=dict(color='rgba(0,0,0,0)'),
                hoverinfo="skip",
                showlegend=False
            ), row=row, col=col)

        # Update subplot axes labels
        figfinal.update_xaxes(title_text="Finetuning data percentage", row=row, col=col)
        figfinal.update_yaxes(title_text="Mean Absolute Error", row=row, col=col)

    # Show the figure
    figfinal.show()
    figfinal.write_html(f"../plots/transfer_comparison/intraseason/{from_building}{from_tower}{from_season}_to_{to_building}{to_tower}{to_season}_mae.html")



    # PLOT PERFORMANCE IMPROVEMENT RATIO

    pir = [[[(seed_rmses[0] - rmse) / seed_rmses[0] for rmse in seed_rmses] for seed_rmses in method_rmses] for method_rmses in rmse_list]

    # Create a subplots figure with two subplots and set title
    figfinal = make_subplots(rows=1, cols=2, subplot_titles=("Finetuning all layers (LSTM+Dense)", "Finetuning output layer only (Dense)"))
    figfinal.update_layout(
        title=f"{from_building} Tower {from_tower} ({from_season}) to {to_building} Tower {to_tower} ({to_season}): Comparison of transfer methods (PIR)",
        showlegend=True
    )

    # Add traces to the subplots
    for tranfer_method_i, tranfer_method in enumerate(["(left)", "(right)"]):
        row = 1
        col = tranfer_method_i + 1
        for s, seed in enumerate(shuffle_seeds):
            x = [fp * 100 for fp in ft_percents]
            # add errors of transfers
            figfinal.add_trace(
                go.Scatter(
                    x=x, 
                    y=pir[tranfer_method_i][s], 
                    name=f"PIR seed-{seed} {tranfer_method}"),
                row=row, col=col
            )
        figfinal.update_xaxes(title_text="Finetuning data percentage", row=row, col=col)
        figfinal.update_yaxes(title_text="Performance improvement ratio", row=row, col=col)

    figfinal.show()
    figfinal.write_html(f"../plots/transfer_comparison/intraseason/{from_building}{from_tower}{from_season}_to_{to_building}{to_tower}{to_season}_pir.html")

# Intra-season (Summer)

In [ ]:
# create all combinations of intraseason transfers for Kissam and ESB
buildings = ["ESB", "Kissam"]
towers = [1, 2]
seasons = ["summer", "fall"]
intraseason_combinations = [((b1,t1,s1),(b2,t2,s2)) for s1 in seasons for t1 in towers for b1 in buildings for s2 in seasons for t2 in towers for b2 in buildings if (b1,t1,s1)!=(b2,t2,s2) and s1==s2]
intraseason_combinations = list(set(intraseason_combinations)) # making sure there are no duplicates

In [ ]:
for c in intraseason_combinations:
    b1 = c[0][0]
    t1 = c[0][1]
    s1 = c[0][2]
    b2 = c[1][0]
    t2 = c[1][1]
    s2 = c[1][2]
    print(c)
    plot_multiseed_error_pir(ft_percents=ft_percents, from_building=b1, from_tower=t1, from_season=s1, to_building=b2, to_tower=t2, to_season=s2)

### Testing on MRB:

In [1]:
# create all combinations of interseason transfers between summer and fall for transferring from Kissam and ESB to MRB
buildings1 = ["ESB", "Kissam"]
buildings2 = ["MRB"]
towers = [1, 2]
seasons = ["summer", "fall"]
intraseason_combinations_to_mrb = [((b1,t1,s1),(b2,t2,s2)) for s1 in seasons for t1 in towers for b1 in buildings1 for s2 in seasons for t2 in towers for b2 in buildings2 if (b1,t1,s1)!=(b2,t2,s2) and s1==s2]
intraseason_combinations_to_mrb = list(set(intraseason_combinations_to_mrb)) # making sure there are no duplicates

In [ ]:
for c in intraseason_combinations_to_mrb:
    b1 = c[0][0]
    t1 = c[0][1]
    s1 = c[0][2]
    b2 = c[1][0]
    t2 = c[1][1]
    s2 = c[1][2]
    print(c)
    # plot_multiseed_error_pir(ft_percents=ft_percents, from_building=b1, from_tower=t1, from_season=s1, to_building=b2, to_tower=t2, to_season=s2)

# Inter-season (Summer to Fall, Fall to Summer)

### Kissam and ESB

In [ ]:
# create all combinations of interseason transfers between summer and fall for Kissam and ESB
buildings = ["ESB", "Kissam"]
towers = [1, 2]
seasons = ["summer", "fall"]
interseason_combinations = [((b1,t1,s1),(b2,t2,s2)) for s1 in seasons for t1 in towers for b1 in buildings for s2 in seasons for t2 in towers for b2 in buildings if (b1,t1,s1)!=(b2,t2,s2) and s1!=s2]
interseason_combinations = list(set(interseason_combinations)) # making sure there are no duplicates

In [ ]:
for c in interseason_combinations:
    b1 = c[0][0]
    t1 = c[0][1]
    s1 = c[0][2]
    b2 = c[1][0]
    t2 = c[1][1]
    s2 = c[1][2]
    print(c)
    plot_multiseed_error_pir(ft_percents=ft_percents, from_building=b1, from_tower=t1, from_season=s1, to_building=b2, to_tower=t2, to_season=s2)

### Testing on MRB

In [ ]:
# create all combinations of interseason transfers between summer and fall for transferring from Kissam and ESB to MRB
buildings1 = ["ESB", "Kissam"]
buildings2 = ["MRB"]
towers = [1, 2]
seasons = ["summer", "fall"]
interseason_combinations_to_mrb = [((b1,t1,s1),(b2,t2,s2)) for s1 in seasons for t1 in towers for b1 in buildings1 for s2 in seasons for t2 in towers for b2 in buildings2 if (b1,t1,s1)!=(b2,t2,s2) and s1!=s2]
interseason_combinations_to_mrb = list(set(interseason_combinations_to_mrb)) # making sure there are no duplicates

In [ ]:
for c in interseason_combinations_to_mrb:
    b1 = c[0][0]
    t1 = c[0][1]
    s1 = c[0][2]
    b2 = c[1][0]
    t2 = c[1][1]
    s2 = c[1][2]
    print(c)
    plot_multiseed_error_pir(ft_percents=ft_percents, from_building=b1, from_tower=t1, from_season=s1, to_building=b2, to_tower=t2, to_season=s2)